<h1>Federated LSTM-Predicting Next Word</h1>

This example demonstrates how predicting next word model in PyTorch could be trained using federated learning with PySyft. We distribute the text data to two workers Bob and Alice to whom the model is sent and trained. Upon training the model the trained model is sent back to the owner of the model and used to make predictions.Federated learning applied to predicting next word helps to improve accuracy of predictor while ensuring the exact text of the end users are not known. 

Hrishikesh Kamath - GitHub: @<a href="http://github.com/kamathhrishi">kamathhrishi</a>

In [1]:
#Import required libraries

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset

In [2]:
#Dataset consisting of a small Shakespere Passage

test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [3]:
import syft as sy

hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

<h1>Federated Dataset</h1>

In [4]:
class TextDataset(Dataset):

    def __init__(self,text,transform=None):
        
        """arguments:
        
             text (List of Strings): Text corpus 
             transform: List of transforms to be performed on the input data
             
        """

        self.text = text
        self.data=[]
        self.targets=[]
        self.transform = transform
        
        #Create Trigrams 
        self.create_context()

    def __len__(self):
        
        return len(self.data)
    
    def create_context(self):
        
        '''Function used to seperate target and context words and convert them to torch tensors'''
        
        context=[]
        
        for i in range(len(self.text)-args.context_size):
            
            vec=[]
            
            for j in range(0,args.context_size):
                
                vec.append(encode(self.text[i+j]))
                
            context.append([torch.tensor(vec,dtype=torch.float),torch.tensor(encode_lol(self.text[i+args.context_size]))])
            
        for words,target in context:
            
            self.data.append(words)
            self.targets.append(target)
            
        return(context)
        

    def __getitem__(self, idx):
        
        sample=self.data[idx]
        target=self.targets[idx]
                
        if self.transform:
            sample = self.transform(sample)

        return sample,target

In [5]:
def encode(word):
    
    """Get Glove Word Vector given a word if present in trained Corpus or else return random vector"""
    
    try: 
        
        return word_vector[word[0].lower()]
    
    except:
        
        return [50]
    

def encode_lol(word):
    
      """Get Index for given word in one hot vector"""
        
      w_class=(word_to_ix[word])
        
      return [w_class]
    
def repackage_hidden(h):
    
    """Wraps hidden states in new Tensors, to detach them from their history."""
    
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

def loadGloveModel(gloveFile):
    
    """Loads glove vectors as Python Dictionary Datatype"""
    
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

def create_context(dataset):
        
        '''Function used to seperate target and context words and convert them to torch tensors'''
        
        context=[]
        
        for i in range(len(dataset)-args.context_size):
            
            vec=[]
            
            for j in range(0,args.context_size):
                
                vec.append(encode(dataset[i+j]))
                
            context.append([torch.tensor(vec,dtype=torch.float),torch.tensor(encode_lol(dataset[i+args.context_size]))])
            
        return(context)

In [6]:
class Arguments():
    
    """Used to define hyperparameters for learning algorithm"""
    
    def __init__(self):
        
        self.batch_size = 1
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.00001
        self.momentum = 0.5 #<-We currenly do not support momentum
        self.no_cuda = False
        self.seed = 2
        self.log_interval = 10
        self.save_model = False
        self.context_size=5
        
args=Arguments()

In [9]:
word_vector=loadGloveModel("glove-global-vectors-for-word-representation/glove.6B.50d.txt")
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

Loading Glove Model
Done. 400000  words loaded!


In [10]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
                         TextDataset(test_sentence)
                         .federate((bob, alice)),batch_size=args.batch_size)

In [11]:
#Define seed to have constancy in the results
torch.manual_seed(args.seed)


<h1>Define Architecture</h1>

In [12]:
class LSTMNext(nn.Module):
    
    """Defines Model of Neural Network"""
    
    def __init__(self,embedding_size,vocab_size,hidden_size):
        
        super(LSTMNext,self).__init__()
        self.embedding=[]
        self.lstm=nn.LSTM(embedding_size,vocab_size,batch_first=False) 
        
    def forward(self,sentance,hidden):
        
        out,hidden=self.lstm(sentance,hidden)
        
        return out,hidden

In [13]:
#Define loss function
loss_function = nn.NLLLoss()

#Define instance of model
model = LSTMNext(50,len(vocab),300)

#Create optimizer instance
optimizer = optim.SGD(model.parameters(), lr=args.lr)

#Retreive Dataset
Dataset=create_context(test_sentence)

<h1>Train Model</h1>

In [14]:
#Initialize Hidden Layers
hidden1=torch.from_numpy(np.random.rand(args.context_size,len(vocab))).view(1,args.context_size,len(vocab))
hidden2=torch.from_numpy(np.random.rand(args.context_size,len(vocab))).view(1,args.context_size,len(vocab))
Hidden=(hidden1,hidden2)

In [15]:
def train():
    
    
    model.train()
    
    loss=0
    
    for context, target in federated_train_loader:
        
        print(context)
        
        model.send(context.location)
        
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()
        
        hidden = repackage_hidden(Hidden)

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        #print(model(context[0]))
        
        print(context.shape)
        print(hidden[0].shape)
        log_probs,hidden=model(context,hidden)
        print(log_probs.shape)
        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        
        target_tensor=target
        
        loss = loss_function(log_probs.view(1,-1),target_tensor)
        
        if(epoch%args.log_interval==0):
            
            print(torch.topk(log_probs,k=3)[1])

        # Step 5. Do the backward pass and update the gradient
        loss.backward(retain_graph=True)
        optimizer.step()
        
        model.get()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        loss += loss.item()
        
    print(loss)
    
    

In [16]:
for epoch in range(args.epochs):
    
    print("EPOCH: ",epoch)
    train()

EPOCH:  0
(Wrapper)>[PointerTensor | me:11957368632 -> bob:72489908847]
torch.Size([1, 5, 50])
torch.Size([1, 5, 97])


RuntimeError: bool value of Tensor with no values is ambiguous